# Tile Generation Benchmarks for Varied Chunk Sizes

## Explanation

In this notebook we compare the performance of tiling artificially generated Zarr data to different chunk sizes. The CMIP6 data provides an excellent real world dataset, but is relatively low resolution. In order to study the impact of higher resolution data, we artificially generated Zarr datastores to explore the relationship between tile generation time and chunk size.

## Setup

In [1]:
# External modules
import hvplot.pandas
import holoviews as hv
import json
import pandas as pd
pd.options.plotting.backend = 'holoviews'
import warnings
warnings.filterwarnings('ignore')

# Local modules
import sys; sys.path.append('..')
import helpers.dataframe as dataframe_helpers
import helpers.eodc_hub_role as eodc_hub_role
from xarray_tile_test import XarrayTileTest

In [2]:
credentials = eodc_hub_role.fetch_and_set_credentials()

Load the fake datasets which have increasingly fine spatial resolution and thus increasingly large chunk size.

In [8]:
# Run 3 iterations of each setting
iterations = 5
zooms = range(6)
all_zarr_datasets = json.loads(open('../01-generate-datasets/fake-datasets.json').read())
zarr_datasets = {k: v for k, v in all_zarr_datasets.items() if 'single_chunk' in k}

## Run Tests

In [10]:
results = []

for zarr_dataset_id, zarr_dataset in zarr_datasets.items():
    zarr_tile_test = XarrayTileTest(
        dataset_id=zarr_dataset_id,
        **zarr_dataset
    )

    # Run it 3 times for each zoom level
    for zoom in zooms:
        zarr_tile_test.run_batch({'zoom': zoom}, batch_size=iterations)

    results.append(zarr_tile_test.store_results(credentials))

Wrote instance data to s3://nasa-eodc-data-store/test-results/20230907012954_XarrayTileTest_single_chunk_store_lat1024_lon2048.zarr.json
Wrote instance data to s3://nasa-eodc-data-store/test-results/20230907013007_XarrayTileTest_single_chunk_store_lat1448_lon2896.zarr.json
Wrote instance data to s3://nasa-eodc-data-store/test-results/20230907013032_XarrayTileTest_single_chunk_store_lat2048_lon4096.zarr.json
Wrote instance data to s3://nasa-eodc-data-store/test-results/20230907013117_XarrayTileTest_single_chunk_store_lat2896_lon5792.zarr.json
Wrote instance data to s3://nasa-eodc-data-store/test-results/20230907013246_XarrayTileTest_single_chunk_store_lat4096_lon8192.zarr.json
Wrote instance data to s3://nasa-eodc-data-store/test-results/20230907013252_XarrayTileTest_single_chunk_store_lat512_lon1024.zarr.json
Wrote instance data to s3://nasa-eodc-data-store/test-results/20230907013258_XarrayTileTest_single_chunk_store_lat724_lon1448.zarr.json


## Read and Plot Results

In [11]:
all_df = dataframe_helpers.load_all_into_dataframe(credentials, results)
expanded_df = dataframe_helpers.expand_timings(all_df)

In [14]:
expanded_df = expanded_df.sort_values('chunk_size_mb')

In [15]:
cmap = ["#E66100", "#5D3A9B"]
plt_opts = {"width": 400, "height": 300}

plts = []

for zoom_level in zooms:
    df_level = expanded_df[expanded_df["zoom"] == zoom_level]
    plts.append(
        df_level.hvplot.box(
            y="time",
            by=["chunk_size_mb"],
            c="chunk_size_mb",
            cmap=cmap,
            ylabel="Time to render (ms)",
            xlabel="Chunk size (MB)",
            legend=False,
            title=f"Zoom level {zoom_level}",
        ).opts(**plt_opts)
    )
hv.Layout(plts).cols(2)

:Layout
   .BoxWhisker.I   :BoxWhisker   [chunk_size_mb]   (time)
   .BoxWhisker.II  :BoxWhisker   [chunk_size_mb]   (time)
   .BoxWhisker.III :BoxWhisker   [chunk_size_mb]   (time)
   .BoxWhisker.IV  :BoxWhisker   [chunk_size_mb]   (time)
   .BoxWhisker.V   :BoxWhisker   [chunk_size_mb]   (time)
   .BoxWhisker.VI  :BoxWhisker   [chunk_size_mb]   (time)

In [16]:
expanded_df.to_csv('results-csvs/03-chunk-size-results.csv')